`open3d_env`

In [1]:
import numpy as np 
from scipy.spatial import KDTree
import trimesh
import glob
import os
from sklearn.decomposition import PCA
import tqdm
import pickle

import sys
sys.path.append('/home/pelissier/These-ATER/Papier_international3/Dataset')  # Adjust the path based on the relative location
from utils import *

In [2]:
dir_mesh_aligned = "/home/pelissier/These-ATER/Papier_international3/Code/Comparaison-User-study/Alignement/Dataset-aligned"

In [ ]:
def read_pkl(dict_path):
    output = open(dict_path,'rb')
    dict = pickle.load(output)
    return dict

def create_rotation_matrix(axis, angle_degrees):
    # Convertir l'angle en radians
    angle_radians = np.radians(angle_degrees)
    # Déterminer le vecteur de direction en fonction de l'axe
    if axis.upper() == 'X':    direction = [1, 0, 0]
    elif axis.upper() == 'Y':  direction = [0, 1, 0]
    elif axis.upper() == 'Z':  direction = [0, 0, 1]
    else: raise ValueError("L'axe doit être 'X', 'Y' ou 'Z'.")
    # Créer la matrice de rotation
    rotation_matrix = trimesh.transformations.rotation_matrix(
        angle=angle_radians,  # Angle en radians
        direction=direction,  # Axe de rotation
        point=[0, 0, 0])  # Centre de rotation (origine)
    return rotation_matrix

## ACP

In [ ]:
def run_acp(mesh, aff= False):
    "Mesh est un Trimesh"
    # Extract vertices
    vertices = np.array(mesh.vertices)
    center = np.mean(vertices, axis=0)

    # Compute the covariance matrix
    cov_matrix = np.cov(vertices.T)

    # Perform eigen decomposition
    eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)

    # Sort eigenvectors by eigenvalues (descending order)
    sorted_indices = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[sorted_indices]
    eigenvectors = eigenvectors[:, sorted_indices]

    # Principal axes are the eigenvectors
    principal_axes = eigenvectors

    # Print results
    if aff:
        print("Eigenvalues:", eigenvalues)
        print("Principal Axes (Eigenvectors):\n", principal_axes)

        # Visualize the principal axes (optional)
        # Each eigenvector is scaled by its eigenvalue for visualization
        for i, axis in enumerate(principal_axes.T):
            print(f"Principal Axis {i + 1}: {axis}")
        
    return eigenvalues, center, eigenvectors

def angle_between_vectors(u, v):
    # Calcul du produit scalaire
    dot_product = np.dot(u, v)
    # Calcul des normes
    norm_u = np.linalg.norm(u)
    norm_v = np.linalg.norm(v)
    # Calcul de l'angle en radians
    cos_theta = dot_product / (norm_u * norm_v)
    # Clamp pour éviter les erreurs dues aux approximations numériques
    cos_theta = np.clip(cos_theta, -1.0, 1.0)
    angle_rad = np.arccos(cos_theta)
    # Conversion en degrés
    angle_deg = np.degrees(angle_rad)
    return angle_deg

def align_principal_axes(mesh1_axes, mesh2_axes, mesh2):
    """
    Align the first principal axes of two meshes while keeping the third axes collinear.
    
    Parameters:
        mesh1_axes (np.ndarray): Principal axes of the first mesh (3x3 matrix).
        mesh2_axes (np.ndarray): Principal axes of the second mesh (3x3 matrix).
        vertices2 (np.ndarray): Vertices of the second mesh (Nx3 matrix).
        
    Returns:
        np.ndarray: Transformed vertices of the second mesh.
    """
    # Extract third principal axes
    v3_1 = mesh1_axes[:, 2]
    v3_2 = mesh2_axes[:, 2]

    # Ensure the third axes are aligned
    angle = abs(angle_between_vectors(v3_1, v3_2))
    if ((angle > 30) and ((angle < 150) or (angle > 210))) :
        raise ValueError("Third principal axes are not collinear. Pre-align them before using this function.")

    # Project the first axis of mesh1 onto the plane orthogonal to v3_1
    v1_1 = mesh1_axes[:, 0]
    v1_1_proj = v1_1 - np.dot(v1_1, v3_1) * v3_1
    v1_1_proj /= np.linalg.norm(v1_1_proj)

    # Get the first axis of mesh2
    v1_2 = mesh2_axes[:, 0]

    # Compute the rotation angle to align v1_2 with v1_1_proj in the plane orthogonal to v3_1
    cos_theta = np.dot(v1_2, v1_1_proj)
    sin_theta = np.linalg.norm(np.cross(v1_2, v1_1_proj))

    # Compute the rotation matrix around v3_1
    v3_cross = np.array([
        [0, -v3_1[2], v3_1[1]],
        [v3_1[2], 0, -v3_1[0]],
        [-v3_1[1], v3_1[0], 0]
    ])
    rotation_matrix = (
        np.eye(3)
        + sin_theta * v3_cross
        + (1 - cos_theta) * np.dot(v3_cross, v3_cross)
    )

    # Apply the rotation to the vertices of the second mesh
    transformed_vertices2 = np.dot(mesh2.vertices, rotation_matrix.T)
    
    # Create a new mesh with rotated vertices
    aligned_mesh2 = trimesh.Trimesh(vertices=transformed_vertices2, faces=mesh2.faces)

    return aligned_mesh2

In [ ]:
paths_mesh_iso = files = glob.glob(os.path.join("/home/pelissier/These-ATER/Papier_international3/Dataset/ModelNet40_remeshing_iso/car/", "**", "*.obj"), recursive=True); print(len(paths_mesh_iso))
#paths_mesh_to_rotate = [path for path in paths_mesh_to_rotate if "0001" in path]; print(len(paths_mesh_to_rotate), paths_mesh_to_rotate)

In [ ]:
## Load the meshes
# Mesh source : par défaut le premier de la liste
path_mesh_source = paths_mesh_iso[0]; print(path_mesh_source)
mesh_source = trimesh.load(path_mesh_source, process=False)
# ACP mesh source
_, _, eigenvectors_source = run_acp(mesh_source, aff=False)
# Save mesh source
directory, name = os.path.split(path_mesh_source)
categorie, type = get_info_path(path_mesh_source)
mesh_source.export(os.path.join(dir_mesh_aligned, categorie, type, name.replace(".obj", "_aligned.obj")))
with open("/home/pelissier/These-ATER/Papier_international3/Code/Comparaison-User-study/Alignement/Dataset-aligned/"+categorie+"/"+categorie+"_source_mesh.txt", "w") as file: file.write(path_mesh_source)
paths_mesh_to_rotate = paths_mesh_iso[1:]

pbl = []
for path_mesh_to_rotate in tqdm.tqdm(paths_mesh_to_rotate[:0]):
    try : 
        # Load the meshes
        mesh2_to_rotate = trimesh.load(path_mesh_to_rotate, process=False)
        num = os.path.basename(path_mesh_to_rotate).split("_")[1]
        # ACP mesh_to_rotate
        _, _, eigenvectors_to_rotate = run_acp(mesh2_to_rotate, aff=False)
        # Align mesh2 to mesh1
        aligned_mesh2 = align_principal_axes(eigenvectors_source, eigenvectors_to_rotate, mesh2_to_rotate)
        # Save the aligned mesh
        directory, name = os.path.split(path_mesh_to_rotate)
        categorie, type = get_info_path(path_mesh_to_rotate)
        aligned_mesh2.export(os.path.join(dir_mesh_aligned, categorie, type, name.replace(".obj", "_aligned.obj")))
        
    except Exception as e:
        pbl.append(path_mesh_to_rotate)
        directory, name = os.path.split(path_mesh_to_rotate)
        categorie, type = get_info_path(path_mesh_to_rotate)
        mesh2_to_rotate.export(os.path.join(dir_mesh_aligned, categorie, type, name.replace(".obj", "_aligned-PBL.obj")))

print(len(pbl), pbl)     

In [ ]:
paths_mesh_iso[0]

Toutes (quasiment) les voitures sont alignées), il faut verifier les orientations devant/derrière

### Debug

In [ ]:
# Load the meshes
path_mesh_source = path_mesh_source
path_mesh_target = [path for path in paths_mesh_to_rotate if "0236" in path][0]; print(path_mesh_target)

In [ ]:
# Load the mesh
mesh_target = trimesh.load(path_mesh_target, process=False)
mesh_source = trimesh.load(path_mesh_source, process=False)

# ACP 
eigenvalues_source, center_source, eigenvectors_source = run_acp(mesh_source, aff=False)
eigenvalues_target, center_target, eigenvectors_target = run_acp(mesh_target, aff=False)

# Create a Trimesh scene
scene = trimesh.Scene()
scene.add_geometry(mesh_source)
scene.add_geometry(mesh_target)

# Scale the axes for visualization
axis_length = np.max(eigenvalues_source) * 6  # Scale factor
scaled_axes_source = eigenvectors_source.T * axis_length

# Add principal axes to the scene
colors = [[255, 0, 0, 255], [0, 255, 0, 255], [0, 0, 255, 255]]  # RGBA for X, Y, Z
for i, axis in enumerate(scaled_axes_source):
    start_point = center_source
    end_point = center_source + axis
    line = trimesh.load_path(np.array([start_point, end_point]))
    line.colors = np.array([colors[i]])  # Assign a single color per line
    scene.add_geometry(line)

# Scale the axes for visualization
axis_length = np.max(eigenvalues_target) * 6  # Scale factor
scaled_axes_target= eigenvectors_target.T * axis_length

# Add principal axes to the scene
colors = [[255, 0, 255, 255], [0, 255, 255, 255], [255, 255, 0, 255]]  # RGBA for X, Y, Z
for i, axis in enumerate(scaled_axes_target):
    start_point = center_target
    end_point = center_target + axis
    line = trimesh.load_path(np.array([start_point, end_point]))
    line.colors = np.array([colors[i]])  # Assign a single color per line
    scene.add_geometry(line)
    
    
## test alignement
try :
    mesh_target_align = align_principal_axes(eigenvectors_source, eigenvectors_target, mesh_target)
    mesh_target_align.apply_translation(np.array([1, 1, 1]))
    scene.add_geometry(mesh_target_align)
except Exception as e: print(e)

# Show the scene
#scene.show()

## Rotation 180°


In [ ]:
# Matrice de rotation de 180° autour de l'axe Z
rotation_matrix = trimesh.transformations.rotation_matrix(
    angle=np.pi,  # 180° en radians
    direction=[0, 0, 1],  # Axe Z
    point=[0, 0, 0]  # Centre de rotation (origine)
)
print(rotation_matrix.shape)

(4, 4)


### ROTATION 180° autour de Z

In [4]:
mesh_to_rotate_180 = read_paths_from_txt('/home/pelissier/These-ATER/Papier_international3/Code/Comparaison-User-study/Alignement/paths/car_meshes_not_aligned.txt'); print(len(mesh_to_rotate_180))
mesh_pbl = read_paths_from_txt('/home/pelissier/These-ATER/Papier_international3/Code/Comparaison-User-study/Alignement/paths/car_meshes_PBL.txt'); print(len(mesh_pbl))
all_mesh_aligned = read_paths_from_txt('//home/pelissier/These-ATER/Papier_international3/Code/Comparaison-User-study/Alignement/paths/paths_aligned_car.txt'); print(len(all_mesh_aligned))

for mesh_path in tqdm.tqdm(all_mesh_aligned[:0]):
    #try:
    if True:
        mesh = trimesh.load(mesh_path.replace('Dataset-aligned', dir_mesh_aligned))
        if mesh_path in mesh_to_rotate_180:
            # # Appliquer la rotation
            # mesh.apply_transform(rotation_matrix)
            # # Sauvegarder le mesh transformé
            # mesh.export(mesh_path.replace('Dataset-aligned', dir_mesh_aligned).replace('.obj', '_ok.obj'))
            # # Sauvegarde transformatison
            # metadata = {'transformations1': rotation_matrix}            
            # with open(mesh_path.replace('Dataset-aligned', dir_mesh_aligned).replace('.obj', '_ok.pkl'), "wb") as f: pickle.dump(metadata, f)
            continue
            
        elif mesh_path in mesh_pbl: continue
            #mesh.export(mesh_path.replace('Dataset-aligned', dir_mesh_aligned).replace('.obj', '_PBL.obj'))
            
        else : continue
            #mesh.export(mesh_path.replace('Dataset-aligned', dir_mesh_aligned).replace('.obj', '_ok.obj'))
            # metadata = {'transformations1': np.eye(4)}            
            # with open(mesh_path.replace('Dataset-aligned', dir_mesh_aligned).replace('.obj', '_ok.pkl'), "wb") as f: pickle.dump(metadata, f)

172
3
297


0it [00:00, ?it/s]


In [7]:
# modele de ref donc Identité et un autre modele
print(read_pkl('/home/pelissier/These-ATER/Papier_international3/Code/Comparaison-User-study/Alignement/Dataset-aligned/car/test/car_0229_SMPLER_centered_scaled_remeshing_iso_iter5_aligned_us_ok.pkl'),
read_pkl('/home/pelissier/These-ATER/Papier_international3/Code/Comparaison-User-study/Alignement/Dataset-aligned/car/train/car_0192_SMPLER_centered_scaled_remeshing_iso_iter6_aligned_us_ok.pkl'))

{'transformations1': array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])} {'transformations1': array([[-1.0000000e+00, -1.2246468e-16,  0.0000000e+00,  0.0000000e+00],
       [ 1.2246468e-16, -1.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  1.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  1.0000000e+00]])}


### Cas particulier

Juste une rotation de 180° autour de Z

In [8]:
## Rotation 180°
num_to_rotate = ['0192', '0181']

paths_mesh_to_rotate = read_paths_from_txt('/home/pelissier/These-ATER/Papier_international3/Code/Comparaison-User-study/Alignement/paths/car_meshes_not_aligned_ok1.txt')

# Matrice de rotation de 180° autour de l'axe Z
for mesh_path in tqdm.tqdm(paths_mesh_to_rotate[:0]):
    mesh = trimesh.load(mesh_path.replace('Dataset-aligned', dir_mesh_aligned))
    # Appliquer la rotation
    mesh.apply_transform(rotation_matrix)
    # Sauvegarder le mesh transformé
    mesh.export(mesh_path.replace('Dataset-aligned', dir_mesh_aligned))
    # metadata = {'transformations1': rotation_matrix}            
    # with open(mesh_path.replace('Dataset-aligned', dir_mesh_aligned).replace('.obj', '_ok.pkl'), "wb") as f: pickle.dump(metadata, f)

0it [00:00, ?it/s]


Regarder test/0236 Train/0021 et 0070

In [9]:
## Rotation 180°
paths_mesh_PBL = read_paths_from_txt('/home/pelissier/These-ATER/Papier_international3/Code/Comparaison-User-study/Alignement/paths/car_meshes_PBL.txt'); print(len(paths_mesh_PBL))
paths_mesh_PBL

3


['Dataset-aligned/car/test/car_0236_SMPLER_centered_scaled_remeshing_iso_iter4_aligned_us-PBL.obj',
 'Dataset-aligned/car/train/car_0021_SMPLER_centered_scaled_remeshing_iso_iter5_aligned_us.obj',
 'Dataset-aligned/car/train/car_0070_SMPLER_centered_scaled_remeshing_iso_iter5_aligned_us.obj']

In [ ]:
mesh_path =  'Dataset-aligned/car/train/car_0070_SMPLER_centered_scaled_remeshing_iso_iter5_aligned_us.obj'
# Appliquer la rotation
mesh = trimesh.load(mesh_path.replace('Dataset-aligned', dir_mesh_aligned))
matrix = create_rotation_matrix('Z', 240)
#mesh.apply_transform(matrix)
# #Sauvegarder le mesh transformé
# mesh.export(mesh_path.replace('Dataset-aligned', dir_mesh_aligned).replace('.obj', '_ok.obj'))
# # Sauvegarde transformatison
# metadata = {'transformations1': matrix}            
# with open(mesh_path.replace('Dataset-aligned', dir_mesh_aligned).replace('.obj', '_ok.pkl'), "wb") as f: pickle.dump(metadata, f)

## Alignement avec US